In [1]:
import os
import datetime
import time
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.loader.dataloader import DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch_geometric.data import Data
import torch_geometric
import networkx as nx

from sklearn.model_selection import train_test_split

from ClusterDatasetTransformer import ClusterDataset
from train_transformer import *
from data_statistics import *

from IPython.display import display

from Transformer import Transformer
from lang import Lang
from LossFunction import Loss

In [2]:
# CUDA Setup
# device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cpu


In [3]:
input_length = 60
max_seq_length = 60
batch_size = 64
max_nodes = 66

In [4]:
converter = Lang(max_nodes)
vocab_size = converter.n_words

In [ ]:
# Load the dataset
hist_folder = "/eos/user/c/czeh/histo/"
data_folder_training = "/eos/user/c/czeh/graph_data/processed"
store_folder_training = "/eos/user/c/czeh/graph_data_trans"
data_folder_test = "/eos/user/c/czeh/graph_data_test/processed"
store_folder_test = "/eos/user/c/czeh/graph_data_trans_test"

model_folder = "/eos/user/c/czeh/model"
dataset_training = ClusterDataset(converter, store_folder_training, data_folder_training, max_nodes=max_nodes, input_length=input_length)
dataset_test = ClusterDataset(converter, store_folder_test, data_folder_test, max_nodes=max_nodes, input_length=input_length)

Processing...


/eos/user/c/czeh/graph_data_trans/raw/data_id_0.pt
[ 1  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23  3  4  5  6  7  8  3  2
  0  0]
[ 1  4  5  6  7  8  3  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23  3  2
  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_1.pt
[ 1  7 13 15  3  4  5  6  8  9 10 11 12 14 16 17  3  2  0  0]
[ 1  4  5  6  8  9 10 11 12 14 16 17  3  7 13 15  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_10.pt
[ 1 10  4  5  6  7  8  9  3  2  0  0  0]
[ 1  4  5  6  7  8  9  3 10  2  0  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_100.pt
[ 1  5  6  8 12 13 16 19 20 22 23 24 25 27 28 29 30 31 32  3  4  7  9 10
 11 14 15 17 18 21 26  3  2  0  0]
[ 1  4  7  9 10 11 14 15 17 18 21 26  3  5  6  8 12 13 16 19 20 22 23 24
 25 27 28 29 30 31 32  3  2  0  0]
[ 1  4  7  9 10 11 14 15 17 18 21 26  3  5  6  8 12 13 16 19 20 22 23 24
 25 27 28 29 30 31 32  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_101.pt
[ 1 25 28 29 30 31 32 36 37 38 39 41 42 43  3  4

/eos/user/c/czeh/graph_data_trans/raw/data_id_127.pt
[ 1  5  6 21  3  4  7  8  9 10 11 12 13 14 15 16 18 19 20 22  3  2  0  0
  0]
[ 1  4  7  8  9 10 11 12 13 14 15 16 18 19 20 22  3  5  6 21  3  2  0  0
  0]
[ 1  5  6 21  3  4  7  8  9 10 11 12 13 14 15 16 18 19 20 22  3  2  0  0
  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_128.pt
[ 1  4  5  6  7  8  9 10 11 12 13 15 18 19 20 23 24 28 29 31 32 36  3 14
 16 17 21 22 25 26 27 30 33 34 35 37 39 40 41 42 43 44 45 46 47 48  3  2
  0  0  0]
[ 1  4  5  6  7  8  9 10 11 12 13 15 18 19 20 23 24 28 29 31 32 36  3 14
 16 17 21 22 25 26 27 30 33 34 35 37 39 40 41 42 43 44 45 46 47 48  3  2
  0  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_129.pt
[ 1  7 13  3  4  5  6  8  9 10 11 12 14  3  2  0  0]
[ 1  4  5  6  8  9 10 11 12 14  3  7 13  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_13.pt
[ 1  4  5  9 10 12 21 24 32 34 37 38 40 41 42 44 45 46 47  3  6  7  8 13
 14 15 16 17 18 19 20 23 25 26 27 28 29 31 33 35 36 43  3  2  

/eos/user/c/czeh/graph_data_trans/raw/data_id_157.pt
[ 1  5  7 17 21 22 23 24 25 26 27 32 34 35  3  4  6  8  9 10 11 12 13 14
 15 16 18 19 20 28 29 30 31 33  3  2  0  0]
[ 1  5  7 17 21 22 23 24 25 26 27 32 34 35  3  4  6  8  9 10 11 12 13 14
 15 16 18 19 20 28 29 30 31 33  3  2  0  0]
[ 1  4  6  8  9 10 11 12 13 14 15 16 18 19 20 28 29 30 31 33  3  5  7 17
 21 22 23 24 25 26 27 32 34 35  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_158.pt
[ 1  6  7  8  9 10 11 12 13 14 15 16 17 18 19  3  4  5  3  2  0  0]
[ 1  4  5  3  6  7  8  9 10 11 12 13 14 15 16 17 18 19  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_159.pt
[ 1  4  5 12  3  6  7  8  9 10 11 13 14 15 16 17 18  3  2  0  0]
[ 1  6  7  8  9 10 11 13 14 15 16 17 18  3  4  5 12  3  2  0  0]
/eos/user/c/czeh/graph_data_trans/raw/data_id_16.pt
[ 1  4  5  6  8  9 11 12 18  3  7 10 14 15 16 19 20 22 23 24  3  2  0  0
  0  0  0]
[ 1  7 10 14 15 16 19 20 22 23 24  3  4  5  6  8  9 11 12 18  3  2  0  0
  0  0  0]
/eos

In [ ]:
train_dl = DataLoader(dataset_training, shuffle=True, batch_size=batch_size)
test_dl = DataLoader(dataset_test, shuffle=True, batch_size=batch_size)

In [ ]:
epochs = 100
d_model = 16
num_heads = 2
num_layers = 4
d_ff = 32
dropout = 0
padding = converter.word2index["<PAD>"]
feature_num = len(dataset_test.model_feature_keys)

# Model, loss, and optimizer
model = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, feature_num, max_nodes, max_seq_length, dropout).to(device)
criterion = Loss(converter)

In [ ]:
# Optionally introduce weight decay
# optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

# Drop Step Size over time
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
#Load Weights if needed
# weights = torch.load("/eos/user/c/czeh/tranformer_2.pt", weights_only=True)
# model.load_state_dict(weights["model_state_dict"])
# optimizer.load_state_dict(weights["optimizer_state_dict"])
# start_epoch = weights["epoch"]

In [ ]:
train_loss_hist = []
val_loss_hist = []

In [ ]:
# https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
# Optionally introduce gradient clipping
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.25)

fig_loss, ax_loss = plt.subplots(1, 1)
fig_loss.set_figwidth(6)
fig_loss.set_figheight(3)

display_loss = display(1, display_id=True)

optimizer.zero_grad()
# Training loop
for epoch in range(1, 101):
    print(f'Epoch: {epoch}')
    
    loss = train(model, converter, optimizer, train_dl, epoch, loss_obj=criterion, device=device)
    train_loss_hist.append(loss)
    
    val_loss, pred, y = test(model, converter, test_dl, epoch, loss_obj=criterion, device=device)
    val_loss_hist.append(val_loss)
    
    ax_loss.clear()
    plot_loss(train_loss_hist, val_loss_hist, ax=ax_loss)
    display_loss.update(fig_loss)
    time.sleep(1)
    
    scheduler.step()
    print(f"Epoch {epoch+1}, LR: {scheduler.get_last_lr()[0]}")

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_figheight(6)
fig.set_figwidth(10)
epochs = len(train_loss_hist)
ax.plot(range(1, epochs+1), moving_average(train_loss_hist, 8), label='train', linewidth=2)
ax.plot(range(1, epochs+1), moving_average(val_loss_hist, 8), label='val', linewidth=2)
ax.set_ylabel("Loss", fontsize=14)
ax.set_xlabel("Epochs", fontsize=14)
ax.set_title("Training and Validation Loss", fontsize=14)
ax.legend()

In [ ]:
date = f"{datetime.now():%Y-%m-%d}"
save_model(model, epoch, optimizer, train_loss_hist, val_loss_hist, model_folder, f"tranformer_date_{date}.pt"):

## Test Full Event

In [ ]:
from EventGrouping import EventGrouping

In [ ]:
model2 = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, feature_num, max_nodes, max_seq_length, dropout).to(device)
weights = torch.load("/eos/user/c/czeh/tranformer_4.pt", weights_only=True)
model2.load_state_dict(weights["model_state_dict"])

In [ ]:
runner = EventGrouping(converter, model2, neighborhood=1, seq_length=input_length)
runner(dataset_test.get(0))

In [ ]:
model_feature_keys = np.array([0,  2,  3,  4,  6,  7, 10, 14, 15, 16, 17, 18, 22, 24, 25, 26, 28, 29])
dataset_training.__getitem__(0)[0][:, model_feature_keys][:, -1]

In [ ]:
dataset_test.get(0).cluster

In [ ]:
dataset_training.node_feature_keys[16]

In [ ]:
G = torch_geometric.utils.to_networkx(dataset_test.get(0), to_undirected=True)

In [ ]:
dataset_training.get(0).x[:, 16]

In [ ]:
fig, ax = plt.subplots(1, 1)
nx.draw(G, with_labels=True, ax=ax)

## Random Tests

In [ ]:
targets = dataset_training.__getitem__(0)[2]
mask = targets != -4
mask.unsqueeze(-1)

In [ ]:
targets[targets[:, -1] != -4, :]

In [ ]:
targets[mask].shape[0]/3

In [ ]:
opts = dataset_training.__getitem__(0)[1]
opts = torch.roll(opts, -1, dims=0)
opts[-1] = 5
opts

In [ ]:
out_mask = opts != -4
opts[out_mask].shape[0]

In [ ]:
targets = torch.reshape(targets[mask], (int(targets[mask].shape[0]/3), 3))

In [ ]:
targets[0, :]